In [ ]:
from ftdb import Ftdb

ec = Ftdb('ec.db')
ec.ftNCBI(['NC_000913.3'])

In [ ]:
import sqlite3

dblst = sqlite3.connect('ec.db')
dblst.row_factory = lambda cursor, row: row[0]
curlst = dblst.cursor()
curlst.execute('select qvalue from qualifiers where qkey = "db_xref" and qvalue LIKE "UniProtKB/Swiss-Prot:%" LIMIT 20')
r = curlst.fetchall()
r 

In [ ]:
term = ''
for a in r:
    a = a.split(':')[1]
    term = term + 'id:{0} OR '.format(a)

term[:-4]

In [ ]:
import requests
base = 'https://www.uniprot.org/uniprot/'
params = {
    'query': term[:-4],
    'format': 'tab',
    'columns': 'id,genes,go'
}
r = requests.get(base, params=params)
r

In [ ]:
r.url

In [ ]:
from io import StringIO
import pandas as pd 

csv = StringIO(r.text)
df = pd.read_csv(csv, sep='\t')
df

In [ ]:
for i,r in df.iterrows():
    entry = str(r['Entry'])
    geneNames = str(r['Gene names']).split(' ')
    go = str(r['Gene ontology (GO)']).split(';')
    print(entry, geneNames, go)

In [ ]:
from ftdb import Ftdb
import sqlite3

dblst = sqlite3.connect('ec.db')
dblst.row_factory = lambda cursor, row: row[0]
curlst = dblst.cursor()
curlst.execute('select qvalue from qualifiers where qkey = "db_xref" and qvalue LIKE "UniProtKB/Swiss-Prot:%"')
r = curlst.fetchall()
dblst.close()
acclst = []
for a in r:
    a = a.split(':')[1]
    acclst.append(a)

ec = Ftdb('ec.db')
ec.ftUniProt(acclst)